实例分割(Instance Segmetation)
===
实例分割不仅仅需要分割出不同类别的物体，对于同一种物体，也需要区别出不同的实例。常用的实例分割数据集包括COCO，PennFund数据集等等。实例分割主要有两类方法，单阶段实例分割与两阶段实例分割。

# 1.两阶段方法-自上而下的基于检测的方法

思路是：首先通过目标检测的方法找出实例所在的区域(bounding box)，再在检测框内进行语义分割，每个分割结果都作为一个不同的实例输出。

![Images](images/03_01_001.jpg)

代表作就是大名鼎鼎的Mask R-CNN了，如下图，总体结构就是Faster R-CNN的两阶段目标检测，box head用来做检测，增加了mask head用来做分割，模型大家都很熟，细节就不再赘述。

![Images](images/03_01_002.jpg)

# 2.两阶段方法-自下而上的基于语义分割

思路是：首先进行像素级别的语义分割，再通过聚类、度量学习等手段区分不同的实例。

![Images](images/03_01_003.jpg)

自下而上的工作并不多，通常的做法都是通过Instance Embedding的做法来做。方法如下：
- 语义分割：首先在第一个阶段做了语义分割，得到了所有的物体mask；
- 像素嵌入：再通过使用一个判别式损失函数来训练网络，网络的优化目标是将图像每个像素投影到n维特征空间后，同属于一个实例的像素尽量靠近，形成一个cluster, 每一个实例对应一个cluster, 不同cluster则尽量远离
- 后处理：最后使用聚类的方法（如mean-shift）来输出不同的实例。

# 3.单阶段方法

单阶段的实例分割方法其实也是受到了单阶段目标检测研究的影响，因此也有两种思路
- 受one-stage, anchot-based检测模型(如YOLO，RetinaNet)启发，代表作有YOLACT和SOLO
- 受anchor-free检测模型(如FCOS)启发，代表作有PolarMask和AdaptIS。

## 3.1.Yolact方法

Yolact主要参照了单阶段检测模型RetinaNet，因此把它归类于单阶段的实例分割。YOLACT将实例分割任务拆分成两个并行的子任务：
- 通过一个Protonet网络, 为每张图片生成k个原型mask
- 对每个实例，预测k个的线性组合系数(Mask Coefficients)
- 最后通过线性组合，生成实例mask，在此过程中，网络学会了如何定位不同位置、颜色和语义实例的mask。

具体网络结构如下:

![Images](images/03_01_004.jpg)

- Backbone：Resnet 101+FPN，与RetinaNet相同；
- Protonet：接在FPN输出的后面，是一个FCN网络，预测得到针对原图的原型mask
- Prediction Head：相比RetinaNet的Head，多了一个Mask Cofficient分支，预测Mask系数，因此输出是4*c+k

![Images](images/03_01_005.jpg)

​此外，论文中还用到了Fast NMS方法，比原有的NMS速度更快，精度减得不多。

## 3.2.PolarMask

## 3.3.SOLO

## 3.4.BlendMask